- used 김태정's model(기세현's model base) 

In [2]:
# !pip install tensorflow
# !pip install keras

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

import tensorflow as tf 

from sklearn.preprocessing import MinMaxScaler,StandardScaler, RobustScaler
from sklearn.metrics import confusion_matrix
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *

from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB


plt.rcParams["figure.figsize"] = (16,10) #Make the plots bigger by default
plt.rcParams["lines.linewidth"] = 2 #Setting the default line width
plt.style.use("ggplot")

In [2]:
model = './model/'
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

#데이터의 최하단 3줄을 표시합니다. 
train.tail(3)

,id,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,...,sensor_24,sensor_25,sensor_26,sensor_27,sensor_28,sensor_29,sensor_30,sensor_31,sensor_32,target
2332,2333,-9.001824,5.985711,-8.146347,-10.902201,5.102105,8.133692,32.877614,-3.017438,-3.174442,...,-7.952857,2.049467,-5.825790,-37.989569,15.014132,1.160272,-11.135889,-7.035763,-0.930067,3
2333,2334,-3.987992,3.011460,-11.949323,-3.810885,16.880234,-5.150117,9.182801,4.960190,-21.002525,...,3.080276,2.054739,-1.052350,-6.019488,-7.075333,-5.826058,-3.989168,14.916905,-12.093426,1
2334,2335,-1.838225,-7.023497,-45.877365,20.026927,4.058551,8.062100,19.083782,-21.881795,-9.106341,...,31.130021,5.121935,-1.003704,-58.953961,-22.095226,-0.898581,1.164833,-21.977991,-13.060285,2


In [3]:
train_X = train.drop(['id','target'],axis=1)
train_y = train.target

In [4]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse = False)
y = ohe.fit_transform(train[['target']])
print(y)

[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]


### Scaling

In [6]:
X = train_X
X = np.array(X)
X = X.reshape(X.shape[0]*X.shape[1], 1)

sc = RobustScaler() # 이상치에 대한 영향 최소화
# sc = StandardScaler()

X = sc.fit_transform(X)
X = X.reshape((-1, 8, 4, 1)) # rnn 하면 바꿔줘야댐
X.shape

(2335, 8, 4, 1)

In [7]:
test = test.drop(['id'], axis=1)
test = np.array(test)
test = test.reshape(test.shape[0]*test.shape[1], 1)
test = sc.fit_transform(test)
test = test.reshape((-1, 8, 4, 1)) # rnn 하면 바꿔줘야댐
target = test
target.shape

(9343, 8, 4, 1)

In [61]:
skf = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=2022)

In [9]:
es = EarlyStopping(monitor = 'val_accuracy', patience = 5, mode = 'auto', verbose = 0)

# Model / Train

In [ ]:
cnn_acc = []
cnn_pred = np.zeros((target.shape[0], 4))
i = 0 # cnt

for i, (tr_idx, val_idx) in enumerate(skf.split(X, train.target)) :
    print(f'{i + 1} Fold Training.....')
    tr_x, tr_y = X[tr_idx], y[tr_idx]
    val_x, val_y = X[val_idx], y[val_idx]

    dim_3_3 = 64
    dim_1_1 = 16

    cnn = Sequential([
                    Conv2D(dim_3_3, (3, 2), padding = "same", activation = 'relu', input_shape = (8, 4, 1)),
                    BatchNormalization(),
                    Conv2D(dim_1_1, (2, 1), padding = "same", activation = 'relu'),
                    MaxPooling2D(2, 2),
                    Conv2D(dim_3_3, (3, 2), padding = "same", activation = 'relu'),
                    BatchNormalization(),
                    Conv2D(dim_1_1, (2, 1), padding = "same", activation = 'relu'),
                    BatchNormalization(),
                    Conv2D(dim_3_3, (4, 3), padding = "same", activation = 'relu'),
                    BatchNormalization(),
                    GlobalAveragePooling2D(),
                    Dense(32, activation = 'leaky_relu'),
                    Dense(4, activation = 'softmax')
                    ])
    ### ModelCheckPoint Fold마다 갱신
    mc = ModelCheckpoint(model + f'cnn_model_{i + 1}.h5', save_best_only = True, monitor = 'val_accuracy', mode = 'auto', verbose = 0)
    
    ### Scheduler 
    reduce_lr = ReduceLROnPlateau( 
        monitor='val_accuracy', # 기준이 될 metrics
        factor=0.2, # lr에 변화를 주는 정도
        patience=5, # 정한 epoch 횟수만큼에 변화가 없으면 lr을 변화시킴 
        min_lr=0.001, # 최소 학습률
        mode='auto', # monitor로 정한 metrics가 감소할지, 증가할지 정함. 
                     # auto, min, max 3가지 중 택 1
    )        
    
    ### 모델 compile
    cnn.compile(optimizer = Adam() , loss = "categorical_crossentropy", metrics=["accuracy"])

    cnn.fit(tr_x, tr_y, validation_data = (val_x, val_y), epochs = 100, batch_size = 32, callbacks = [es, mc, reduce_lr], verbose = 0)

    ### 최고 성능 기록 모델 Load
    best = load_model(model + f'cnn_model_{i + 1}.h5')
    ### validation predict
    val_pred = best.predict(val_x)
    ### 확률값 중 최대값을 클래스로 매칭
    val_cls = np.argmax(val_pred, axis = 1)
    ### Fold별 정확도 산출
    fold_cnn_acc = accuracy_score(np.argmax(val_y, axis = 1), val_cls)
    cnn_acc.append(fold_cnn_acc)
    print(f'{i + 1} Fold ACC of cnn = {fold_cnn_acc}\n')

    ### Fold별 test 데이터에 대한 예측값 생성 및 앙상블
    fold_pred = best.predict(target) / skf.get_n_splits()
    cnn_pred += fold_pred

1 Fold Training.....
2 Fold ACC of cnn = 0.8034188034188035

3 Fold Training.....
3 Fold ACC of cnn = 0.8504273504273504

4 Fold Training.....
4 Fold ACC of cnn = 0.8418803418803419

5 Fold Training.....
5 Fold ACC of cnn = 0.8247863247863247

6 Fold Training.....
6 Fold ACC of cnn = 0.8626609442060086

7 Fold Training.....
7 Fold ACC of cnn = 0.7982832618025751

8 Fold Training.....
8 Fold ACC of cnn = 0.8454935622317596

9 Fold Training.....
9 Fold ACC of cnn = 0.8240343347639485

10 Fold Training.....
11 Fold ACC of cnn = 0.811965811965812

12 Fold Training.....
12 Fold ACC of cnn = 0.8162393162393162

13 Fold Training.....
13 Fold ACC of cnn = 0.8547008547008547

14 Fold Training.....
14 Fold ACC of cnn = 0.7991452991452992

15 Fold Training.....
15 Fold ACC of cnn = 0.8290598290598291

16 Fold Training.....
16 Fold ACC of cnn = 0.8626609442060086

17 Fold Training.....
17 Fold ACC of cnn = 0.8197424892703863

18 Fold Training.....
18 Fold ACC of cnn = 0.8369098712446352

19 Fold T

In [54]:
# 상위 11개 출력
# print(i) # 99
num = range(1, i+2)
dict_cnn_acc = dict(zip(num, cnn_acc))
max11 = sorted(dict_cnn_acc.items(), key = lambda item: item[1], reverse = True)
max11[:11] # (model 번호, accuracy)

[(46, 0.8884120171673819),
 (56, 0.8798283261802575),
 (50, 0.871244635193133),
 (94, 0.8675213675213675),
 (72, 0.8632478632478633),
 (75, 0.8632478632478633),
 (6, 0.8626609442060086),
 (16, 0.8626609442060086),
 (70, 0.8626609442060086),
 (35, 0.8589743589743589),
 (44, 0.8589743589743589)]

# Inference

In [55]:
model1 = load_model(model + 'cnn_model_'+ str(max11[0][0]) +'.h5')
model2 = load_model(model + 'cnn_model_'+ str(max11[1][0]) +'.h5')
model3 = load_model(model + 'cnn_model_'+ str(max11[2][0]) +'.h5')
model4 = load_model(model + 'cnn_model_'+ str(max11[3][0]) +'.h5')
model5 = load_model(model + 'cnn_model_'+ str(max11[4][0]) +'.h5')
model6 = load_model(model + 'cnn_model_'+ str(max11[5][0]) +'.h5')
model7 = load_model(model + 'cnn_model_'+ str(max11[6][0]) +'.h5')
model8 = load_model(model + 'cnn_model_'+ str(max11[7][0]) +'.h5')
model9 = load_model(model + 'cnn_model_'+ str(max11[8][0]) +'.h5')
model10 = load_model(model + 'cnn_model_'+ str(max11[9][0]) +'.h5')
model11 = load_model(model + 'cnn_model_'+ str(max11[10][0]) +'.h5')

## validation predict
val_pred1 = model1.predict(X)
val_pred2 = model2.predict(X)
val_pred3 = model3.predict(X)
val_pred4 = model4.predict(X)
val_pred5 = model5.predict(X)
val_pred6 = model6.predict(X)
val_pred7 = model7.predict(X)
val_pred8 = model8.predict(X)
val_pred9 = model9.predict(X)
val_pred10 = model10.predict(X)
val_pred11 = model11.predict(X)
### 확률값 중 최대값을 클래스로 매칭
val_cls1 = np.argmax(val_pred1, axis = 1)
val_cls2 = np.argmax(val_pred2, axis = 1)
val_cls3 = np.argmax(val_pred3, axis = 1)
val_cls4 = np.argmax(val_pred4, axis = 1)
val_cls5 = np.argmax(val_pred5, axis = 1)
val_cls6 = np.argmax(val_pred6, axis = 1)
val_cls7 = np.argmax(val_pred7, axis = 1)
val_cls8 = np.argmax(val_pred8, axis = 1)
val_cls9 = np.argmax(val_pred9, axis = 1)
val_cls10 = np.argmax(val_pred10, axis = 1)
val_cls11 = np.argmax(val_pred11, axis = 1)

### Fold별 정확도 산출
test_cnn_acc = []

fold_cnn_acc = accuracy_score(np.argmax(y, axis = 1), val_cls1)
test_cnn_acc.append(fold_cnn_acc)
print(f'{str(max11[0][0])} Fold ACC of cnn = {fold_cnn_acc}\n')
fold_cnn_acc = accuracy_score(np.argmax(y, axis = 1), val_cls2)
test_cnn_acc.append(fold_cnn_acc)
print(f'{str(max11[1][0])} Fold ACC of cnn = {fold_cnn_acc}\n')
fold_cnn_acc = accuracy_score(np.argmax(y, axis = 1), val_cls3)
test_cnn_acc.append(fold_cnn_acc)
print(f'{str(max11[2][0])} Fold ACC of cnn = {fold_cnn_acc}\n')
fold_cnn_acc = accuracy_score(np.argmax(y, axis = 1), val_cls4)
test_cnn_acc.append(fold_cnn_acc)
print(f'{str(max11[3][0])} Fold ACC of cnn = {fold_cnn_acc}\n')
fold_cnn_acc = accuracy_score(np.argmax(y, axis = 1), val_cls5)
test_cnn_acc.append(fold_cnn_acc)
print(f'{str(max11[4][0])} Fold ACC of cnn = {fold_cnn_acc}\n')
fold_cnn_acc = accuracy_score(np.argmax(y, axis = 1), val_cls6)
test_cnn_acc.append(fold_cnn_acc)
print(f'{str(max11[5][0])} Fold ACC of cnn = {fold_cnn_acc}\n')
fold_cnn_acc = accuracy_score(np.argmax(y, axis = 1), val_cls7)
test_cnn_acc.append(fold_cnn_acc)
print(f'{str(max11[6][0])} Fold ACC of cnn = {fold_cnn_acc}\n')
fold_cnn_acc = accuracy_score(np.argmax(y, axis = 1), val_cls8)
test_cnn_acc.append(fold_cnn_acc)
print(f'{str(max11[7][0])} Fold ACC of cnn = {fold_cnn_acc}\n')
fold_cnn_acc = accuracy_score(np.argmax(y, axis = 1), val_cls9)
test_cnn_acc.append(fold_cnn_acc)
print(f'{str(max11[8][0])} Fold ACC of cnn = {fold_cnn_acc}\n')
fold_cnn_acc = accuracy_score(np.argmax(y, axis = 1), val_cls10)
test_cnn_acc.append(fold_cnn_acc)
print(f'{str(max11[9][0])} Fold ACC of cnn = {fold_cnn_acc}\n')
fold_cnn_acc = accuracy_score(np.argmax(y, axis = 1), val_cls11)
test_cnn_acc.append(fold_cnn_acc)
print(f'{str(max11[10][0])} Fold ACC of cnn = {fold_cnn_acc}\n')


### Fold별 test 데이터에 대한 예측값 생성 및 앙상블
fold_pred = best.predict(target) / 5
cnn_pred += fold_pred

46 Fold ACC of cnn = 0.95203426124197

56 Fold ACC of cnn = 0.9498929336188436

50 Fold ACC of cnn = 0.9584582441113491

94 Fold ACC of cnn = 0.9481798715203427

72 Fold ACC of cnn = 0.9486081370449678

75 Fold ACC of cnn = 0.9340471092077087

6 Fold ACC of cnn = 0.9430406852248394

16 Fold ACC of cnn = 0.9267665952890792

70 Fold ACC of cnn = 0.9623126338329765

35 Fold ACC of cnn = 0.9691648822269807

44 Fold ACC of cnn = 0.9618843683083512



In [56]:
# 몇 번째 모델이 결과가 좋을까? 
print(test_cnn_acc.index(np.max(test_cnn_acc)) + 1)
print(np.max(test_cnn_acc))

10
0.9691648822269807


In [57]:
model1 = load_model('./model/cnn_model_35.h5') # acc 높은 모델 지정

val_pred1 = model1.predict(X)
val_cls1 = np.argmax(val_pred1, axis = 1)

pred_cnn_acc = []

# train['target']과 학습 결과 비교
fold_cnn_acc = accuracy_score(np.argmax(y, axis = 1), val_cls1) 
pred_cnn_acc.append(fold_cnn_acc)
print(f'Fold ACC of cnn = {fold_cnn_acc}\n')

cnn_pred = np.zeros((target.shape[0], 4))
fold_pred = model1.predict(target)
cnn_pred += fold_pred

submission = pd.read_csv('./data/sample_submission.csv')
submission['target'] = np.argmax(cnn_pred, axis = 1)
submission.target.value_counts()

Fold ACC of cnn = 0.9691648822269807



3    2580
0    2269
2    2248
1    2246
Name: target, dtype: int64

In [58]:
submission.to_csv('./result/submission_CNNmodel_35.csv', index = False)

In [59]:
submission.head(10)

,id,target
0,1,0
1,2,0
2,3,1
3,4,3
4,5,2
5,6,0
6,7,0
7,8,1
8,9,3
9,10,3


In [60]:
submission.tail(10)

,id,target
9333,9334,0
9334,9335,2
9335,9336,2
9336,9337,2
9337,9338,1
9338,9339,3
9339,9340,1
9340,9341,0
9341,9342,0
9342,9343,3
